In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from random import uniform
import time
import json
import pandas as pd
import sqlite3
import os

In [2]:
def load_page():
    search = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.ID, 'OuterDiv'))
    )

    results = driver.find_elements_by_xpath('//*[@id="OuterDiv"]/div')
    slots = int(results[-1].text.split()[-1])
    return slots

In [3]:
def fill_out_forms():
    eligible = 'https://am-i-eligible.covid19vaccine.health.ny.gov'
    driver.get(eligible)
    
    load = '//*[@id="statePods_table"]/tbody/tr[1]/td[1]'
    
    loaded = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, load))
    )
    time.sleep(5)
    
    #scrolls down page a bit
    driver.execute_script("window.scrollTo(800, document.body.scrollHeight);")
    time.sleep(2)
    
    #click get started button to go to next page
    get_started = '//*[@id="main"]/section/div/div/div/button/div'
    gs_button = driver.find_element_by_xpath(get_started)
    gs_button.click()
    time.sleep(1)
    
    #path to enter dob
    dob = '//*[@id="relationship"]/div/div[1]/div[1]/div/div[1]/div[1]/input'
    
    #input dob
    dob_input = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, dob))
    )
    dob_input.click()
    dob_input.clear()
    time.sleep(1)
    dob_input.send_keys('03201935')
    dob_input.send_keys(Keys.RETURN)
    time.sleep(1)
    
    #path to sex option
    nb = '//*[@id="relationship"]/div/div[1]/div[2]/fieldset/div/div[3]/input'
    
    #click non-binary
    nb_box = driver.find_element_by_xpath(nb)
    nb_box.click()
    
    time.sleep(1)
    
    #path to live option
    live = '//*[@id="relationship"]/div/div[2]/div[1]/fieldset/div/div[1]/input'
    
    #click yes
    live_box = driver.find_element_by_xpath(live)
    live_box.click()
    
    time.sleep(1)
    
    #path to work option
    work = '//*[@id="relationship"]/div/div[2]/div[2]/fieldset/div/div[2]/input'
    
    #click no
    work_box = driver.find_element_by_xpath(work)
    work_box.click()
    
    time.sleep(1)
    
    #path to enter zip
    zipcode = '//*[@id="relationship"]/div/div[3]/div/div/div[1]/div[1]/input'
    
    #input dob
    zip_input = driver.find_element_by_xpath(zipcode)
    zip_input.click()
    zip_input.clear()
    time.sleep(1)
    zip_input.send_keys('11554')
    zip_input.send_keys(Keys.RETURN)
    time.sleep(15)
    
    ack = '//*[@id="acknowledgement"]/div/section/fieldset/legend/label'
    
    acker = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, ack))
    )
    
    #scrolls down page a bit
    driver.execute_script("window.scrollTo(700, document.body.scrollHeight);")
    time.sleep(2)
    
    #path to consent option
    consent = '//*[@id="acknowledgement"]/div/section/fieldset/div/div/div/input'
    
    #click
    consent_box = driver.find_element_by_xpath(consent)
    consent_box.click()
    
    #path to submit option
    submit = '//*[@id="content"]/section/div[3]/div[2]/button/div'     
    
    try:
        submit_box = driver.find_element_by_xpath(submit)
        submit_box.click()

    except:
        #click
        time.sleep(1)
        submit_box.click()
    
    time.sleep(15)
    
    #path to providers button
    providers = '//*[@id="content"]/section/div[3]/div/button[2]/div'
    
    #click
    try:
        providers_box = driver.find_element_by_xpath(providers)
        providers_box.click()

    except:
        #click
        time.sleep(15)
        providers_box.click()
    
    time.sleep(1)
    
    #path to jb option
    jb = '//*[@id="content"]/section/section[1]/section/section[1]/div[2]/div/p[2]/span/a'
    sb = '//*[@id="content"]/section/section[5]/section/section[1]/div[2]/div/p[2]/span/a'
    
    try:
        #click
        jb_box = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, jb))
        )
        time.sleep(10)
        jb_box.click()
    except:
        time.sleep(5)
        jb_box.click()
    
    time.sleep(1)
    
    driver.switch_to.window(driver.window_handles[1])

In [4]:
got_slot = False

# datetime object containing current date and time
now = datetime.now() # current date and time
today = now.strftime("%B %d")

while got_slot == False:
    # path to chromedriver
    PATH = '/Users/Jason/Documents/git/Webdrivers/chromedriver'
    #opens an instance of chrome
    driver = webdriver.Chrome(PATH)
    time.sleep(5)
    try:
        fill_out_forms()
    except:
        driver.quit()
        continue
    
    for i in range(1000):
        sleep = uniform(2,4)
        #print(i)
        #opens gov site
        #jb = 'https://apps2.health.ny.gov/doh2/applinks/cdmspr/2/counties?OpID=50502584'
        #sb = 'https://apps4.health.ny.gov/doh2/applinks/cdmspr/2/counties?OpID=50503446'
        #jb = 'https://apps7.health.ny.gov/doh2/applinks/cdmspr/2/counties?OpID=BB4FFAE4EC88049EE0530A6C7C168878'
        #driver.get(jb)
        driver.refresh()

        try:
            slots = load_page()
        
        except:
            time.sleep(sleep)
            if i == 999:
                os.system('say "Restart"')
            continue
            
        works = False
        for slot in range(slots):
            #if slot in (0,7,14):
                #continue
            try:
                valid = driver.find_element_by_xpath('//*[@id="Marker{}Div"]/div[2]/div/div[2]/div[8]/div/button'.format(slot))
                month = driver.find_element_by_xpath('//*[@id="Marker{}Div"]/div[2]/div/div[2]/div[1]'.format(slot))
                if (valid.text == 'Event Full') or (today in month.text):
                    continue
                elif month.text.split()[1] in ['February', 'March']:
                    valid.click()
                    works = True
                    break
                else:
                    continue
            except:
                print('exception',slot)
    
        if works == False:
            time.sleep(sleep)
            if i == 999:
                os.system('say "Restart"')
            continue
        else:
            try:
                tic = '//*[@id="content"]/div[4]/div[2]/div/div/table/tbody/tr[2]/td[3]/input'
                tic_box = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, tic))
                )
                tic_box.click()
    
                next_bttn = driver.find_element_by_xpath('//*[@id="content"]/div[4]/div[2]/div/div/div[2]/fieldset/button[2]')
                next_bttn.click()
                page = True
            except:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                #path to jb option
                jb = '//*[@id="content"]/section/section[1]/section/section[1]/div[2]/div/p[2]/span/a'
                sb = '//*[@id="content"]/section/section[5]/section/section[1]/div[2]/div/p[2]/span/a'
    
                try:
                    #click
                    jb_box = WebDriverWait(driver, 30).until(
                        EC.presence_of_element_located((By.XPATH, jb))
                    )
                    time.sleep(10)
                    jb_box.click()
                except:
                    time.sleep(5)
                    jb_box.click()
    
                time.sleep(1)
    
                driver.switch_to.window(driver.window_handles[1])
                page = False
        if page:
            os.system('say "Vaccine time."')
            got_slot = True
            break
        elif i == 999:
            os.system('say "Restart"')
            
        time.sleep(sleep) 
    
    if got_slot:
        break
    
    driver.quit()   
    time.sleep(10) 